# Worldwide Autism Burden

In [1]:
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Importing data

In [2]:
prevalence_abs = pd.read_csv('raw_data/prevalence_abs.csv').drop(['measure', 'metric', 'cause', 'upper', 'lower'], axis=1).rename({'val': 'prevalence_abs'}, axis=1)
prevalence_pct = pd.read_csv('raw_data/prevalence_pct.csv').drop(['measure', 'metric', 'cause', 'upper', 'lower'], axis=1).rename({'val': 'prevalence_pct'}, axis=1)
prevalence_rate = pd.read_csv('raw_data/prevalence_rate.csv').drop(['measure', 'metric', 'cause', 'upper', 'lower'], axis=1).rename({'val': 'prevalence_rate'}, axis=1)

In [3]:
prevalence_abs = prevalence_abs.reset_index().pivot(index=['location', 'age', 'year'], columns=['sex'], values='prevalence_abs').reset_index().rename({'Female': 'female', 'Male': 'male'}, axis=1)
prevalence_pct = prevalence_pct.reset_index().pivot(index=['location', 'age', 'year'], columns=['sex'], values='prevalence_pct').reset_index().rename({'Female': 'female', 'Male': 'male'}, axis=1)
prevalence_rate = prevalence_rate.reset_index().pivot(index=['location', 'age', 'year'], columns=['sex'], values='prevalence_rate').reset_index().rename({'Female': 'female', 'Male': 'male'}, axis=1)



### Cleaning data

In [4]:
prevalence_abs['age'] = prevalence_abs['age'].str.replace(' years', '')
prevalence_pct['age'] = prevalence_pct['age'].str.replace(' years', '')
prevalence_rate['age'] = prevalence_rate['age'].str.replace(' years', '')

### Exploratory Analysis